In [ ]:
import numpy as np
from read_tif import read_tif
from write_tif import write_tif


def calc_ndvi(red: np.ndarray, nir: np.ndarray) -> np.ndarray:
    """
    Normalized Difference Vegetation Index (NDVI).
    
    NDVI = (NIR - Red) / (NIR + Red)
    
    Parameters
    ----------
    red : ndarray
        Red band array.
    nir : ndarray
        Near-infrared band array.
    
    Returns
    -------
    ndarray
        NDVI values in range [-1, 1].
    """
    red = red.astype(float)
    nir = nir.astype(float)
    ndvi = (nir - red) / (nir + red + 1e-6)  # avoid division by zero
    return ndvi


def calc_ndwi(green: np.ndarray, nir: np.ndarray) -> np.ndarray:
    """
    Normalized Difference Water Index (NDWI).
    
    NDWI = (Green - NIR) / (Green + NIR)
    """
    green = green.astype(float)
    nir = nir.astype(float)
    ndwi = (green - nir) / (green + nir + 1e-6)
    return ndwi


def calc_evi(blue: np.ndarray, red: np.ndarray, nir: np.ndarray,
             G=2.5, C1=6.0, C2=7.5, L=1.0) -> np.ndarray:
    """
    Enhanced Vegetation Index (EVI).
    
    EVI = G * (NIR - Red) / (NIR + C1*Red - C2*Blue + L)
    
    Parameters
    ----------
    blue, red, nir : ndarray
        Spectral bands.
    G, C1, C2, L : float
        Coefficients (default MODIS values).
    """
    blue, red, nir = blue.astype(float), red.astype(float), nir.astype(float)
    evi = G * (nir - red) / (nir + C1 * red - C2 * blue + L + 1e-6)
    return evi

In [ ]:
# ========= Example usage =========
if __name__ == "__main__":
    # Example: Landsat (band order may differ by sensor, adjust accordingly)
    # Landsat 8/9: Red = B4, Green = B3, Blue = B2, NIR = B5

    data, X, Y, geo, proj, bands = read_tif("landsat_example.tif")

    # Assume `data` has shape (bands, rows, cols)
    red = data[3]   # B4
    green = data[2] # B3
    blue = data[1]  # B2
    nir = data[4]   # B5

    # Compute indices
    ndvi = calc_ndvi(red, nir)
    ndwi = calc_ndwi(green, nir)
    evi = calc_evi(blue, red, nir)

    # Save outputs
    write_tif(ndvi, geo, proj, "ndvi.tif")
    write_tif(ndwi, geo, proj, "ndwi.tif")
    write_tif(evi, geo, proj, "evi.tif")

    print("Saved: ndvi.tif, ndwi.tif, evi.tif")